# 1. Example of graph modification attack using GRB

GRB provides a unified evaluation scenario for fair comparisons between attacks and defenses. The scenario is **Black-box, Evasion, Inductive, Modification**. Take the case of a citation-graph classification system for example. The platform collects labeled data from previous papers and trains a GNN model. When a batch of new papers are submitted, it updates the graph and uses the trained model to predict labels for them.

* **Black-box**: Both the attacker and the defender have no knowledge about the applied methods each other uses.
* **Evasion**: GNNs are already trained in trusted data (e.g. authenticated users), which are untouched by the attackers but might have natural noises. Thus, attacks will only happen during the inference phase.
* **Inductive**: GNNs are used to classify unseen data (e.g. new users), i.e. validation or test data are unseen during training, which requires GNNs to generalize to out of distribution data.
* **Modification**: The attackers can perturb existing edges or modify node features within constraints. 

In [2]:
import os
import torch
import numpy as np
import scipy.sparse as sp
import grb.utils as utils

## 1.1. Load Dataset

GRB datasets are named by the prefix *grb-*. There are four *mode* ('easy', 'medium', 'hard', 'full') for test set, representing different average degrees of test nodes, thus different difficulty for attacking them. The node features are processed by *arctan* normalization (first standardization then arctan function), which makes node features fall in the same scale.

In [4]:
from grb.dataset import Dataset

dataset_name = 'grb-cora'
dataset = Dataset(name=dataset_name, 
                  data_dir="../../data/",
                  mode='full',
                  feat_norm='arctan')

Dataset 'grb-cora' loaded.
    Number of nodes: 2680
    Number of edges: 5148
    Number of features: 302
    Number of classes: 7
    Number of train samples: 1608
    Number of val samples: 268
    Number of test samples: 804
    Dataset mode: full
    Feature range: [-0.9406, 0.9430]


In [5]:
device = "cuda:0"
adj = dataset.adj
features = dataset.features
labels = dataset.labels
num_features = dataset.num_features
num_classes = dataset.num_classes
test_mask = dataset.test_mask

### 1.1.1 Train surrogate model

In [6]:
from grb.model.torch import GCN
from grb.utils.normalize import GCNAdjNorm

model_name = "gcn"
model_sur = GCN(in_features=dataset.num_features,
                out_features=dataset.num_classes,
                hidden_features=64, 
                n_layers=2,
                adj_norm_func=GCNAdjNorm,
                layer_norm=False,
                residual=False,
                dropout=0.5)
print(model_sur)

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


In [33]:
save_dir = "./saved_models/{}/{}".format(dataset_name, model_name)
save_name = "model_sur.pt"
device = "cuda:0"
feat_norm = None
train_mode = "inductive"  # "transductive"

In [34]:
from grb.trainer.trainer import Trainer

trainer = Trainer(dataset=dataset, 
                  optimizer=torch.optim.Adam(model_sur.parameters(), lr=0.01),
                  loss=torch.nn.functional.cross_entropy,
                  lr_scheduler=False,
                  early_stop=True,
                  early_stop_patience=500,
                  feat_norm=feat_norm,
                  device=device)

In [35]:
trainer.train(model=model_sur, 
              n_epoch=2000,
              eval_every=1,
              save_after=0,
              save_dir=save_dir,
              save_name=save_name,
              train_mode=train_mode,
              verbose=False)

  0%|          | 0/2000 [00:00<?, ?it/s]

Training early stopped. Best validation score: 0.8433
Training runtime: 1.8627.


In [36]:
# by trainer
test_score = trainer.evaluate(model_sur, dataset.test_mask)
print("Test score of surrogate model: {:.4f}".format(test_score))

Test score of surrogate model: 0.8197


## 1.2. Graph Modification Attack

### 1.2.1. DICE (Delete Internally Connect Externally)

In [11]:
from grb.attack.modification.dice import DICE

In [27]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.3
n_edge_mod = int(n_edge_test * n_mod_ratio)
ratio_delete = 0.6

In [28]:
attack = DICE(n_edge_mod, ratio_delete)

In [29]:
adj_attack = attack.attack(adj, dataset.index_test, labels)

Delete internally......


  0%|          | 0/2604 [00:00<?, ?it/s]

Connect externally......


  0%|          | 0/804 [00:00<?, ?it/s]

DICE attack finished. 468 edges were removed, 313 edges were connected.


### 1.2.2. FGA (Fast Gradient Attack)

In [15]:
from grb.attack.modification.fga import FGA

In [16]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.1
n_edge_mod = int(n_edge_test * n_mod_ratio)

In [12]:
attack = FGA(n_edge_mod, device=device)

In [14]:
adj_attack = attack.attack(model_sur, adj, features, dataset.index_test)

  0%|          | 0/2680 [00:00<?, ?it/s]

FGA attack finished. 261 edges were flipped.


### 1.2.3. FLIP (Flipping)

In [17]:
from grb.attack.modification.flip import FLIP

In [18]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.01
n_edge_mod = int(n_edge_test * n_mod_ratio)

In [11]:
# degree flipping
attack = FLIP(n_edge_mod, flip_type="deg", mode="descend", device=device)

In [12]:
# betweenness flipping
attack = FLIP(n_edge_mod, flip_type="bet", mode="ascend", device=device)

In [13]:
# eigen flipping
attack = FLIP(n_edge_mod, flip_type="eigen", mode="descend", device=device)

In [ ]:
adj_attack = attack.attack(adj, dataset.index_test)

In [6]:
from grb.attack.modification.rand import RAND

In [10]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.1
n_edge_mod = int(n_edge_test * n_mod_ratio)
print(n_edge_mod)

260


In [11]:
attack = RAND(n_edge_mod)

In [12]:
adj_attack = attack.attack(adj, dataset.index_test)

  0%|          | 0/260 [00:00<?, ?it/s]

RAND attack finished. 260 edges were randomly flipped.


### 1.2.4. NEA (Network Embedding Attack)

In [19]:
from grb.attack.modification.nea import NEA

In [20]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.1
n_edge_mod = int(n_edge_test * n_mod_ratio)

In [18]:
attack = NEA(n_edge_mod)

In [19]:
adj_attack = attack.attack(adj, dataset.index_test)

  0%|          | 0/260 [00:00<?, ?it/s]

NEA attack finished. 260 edges were flipped.


### 1.2.5. STACK (STricted black-box AttaCK)

In [23]:
from grb.attack.modification.stack import STACK

In [24]:
n_edge_test = adj[test_mask].getnnz()
n_mod_ratio = 0.01
n_edge_mod = int(n_edge_test * n_mod_ratio)

In [25]:
attack = STACK(n_edge_mod)

In [26]:
adj_attack = attack.attack(adj, dataset.index_test)

  0%|          | 0/2604 [00:00<?, ?it/s]

STACK attack finished. 26 edges were flipped.


## 1.3. Evaluation

In [38]:
model_name = "gcn"
save_dir = "./saved_models/{}/{}".format(dataset_name, model_name)
save_name = "model.pt"
model = torch.load(os.path.join(save_dir, save_name))
model = model.to(device)
model.eval()

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)

In [39]:
# original
test_score = utils.evaluate(model, 
                            features=dataset.features,
                            adj=dataset.adj,
                            labels=dataset.labels,
                            feat_norm=model.feat_norm,
                            adj_norm_func=model.adj_norm_func,
                            mask=dataset.test_mask,
                            device=device)
print("Test score: {:.4f}".format(test_score))

Test score: 0.8259


In [40]:
# after attack
test_score = utils.evaluate(model, 
                            features=dataset.features,
                            adj=adj_attack,
                            labels=dataset.labels,
                            feat_norm=model.feat_norm,
                            adj_norm_func=model.adj_norm_func,
                            mask=dataset.test_mask,
                            device=device)
print("Test score: {:.4f}".format(test_score))

Test score: 0.7388
